Source: https://www.kaggle.com/datagov/usa-names/data
<br>Example Code from: https://www.kaggle.com/diamazov/unique-generation-names

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly as plo
import seaborn as sns

In [63]:
names = pd.read_csv('../data/usnames/names_per_year.csv')
del names['Unnamed: 0']

In [64]:
names.head()

,year,gender,name,year_total
0,1921,F,Ah,5
1,1915,F,Ah,5
2,2000,F,Ai,5
3,2002,F,Ai,7
4,1995,F,Ai,5


In [66]:
names['len'] = names['name'].apply(lambda x: len(x))

In [70]:
names

,year,gender,name,year_total,len
0,1921,F,Ah,5,2
1,1915,F,Ah,5,2
2,2000,F,Ai,5,2
3,2002,F,Ai,7,2
4,1995,F,Ai,5,2
5,1980,F,Ai,6,2
6,2004,F,Ai,6,2
7,2009,F,Ai,5,2
8,1971,F,Al,5,2
9,1998,F,An,11,2


In [152]:
yr2016 = names.groupby(names.year).mean()
yr2017 = names.loc[names['year'] == 2017].groupby(names.year).mean()

ValueError: Grouper for '<class 'pandas.core.frame.DataFrame'>' not 1-dimensional

In [150]:
yr2016

,year_total,len
year,,
1910,277.965070,5.870423
1911,310.891209,5.892308
1912,373.895158,5.905684
1913,394.697237,5.957790
1914,425.023332,5.990799
1915,475.257593,5.982002
1916,485.731648,5.985318
1917,495.777184,6.022442
1918,518.371267,6.007209


In [ ]:
check